In [4]:
import numpy as np
import random
from math import log

# Generate random prime
nbits = 5
randPrime = random_prime(2^(nbits), false, 2^(nbits-1))
print(randPrime)

# Parameters
dim = 3
samples = 5
expError = 2
q = randPrime
m = samples
n = dim



M1Test = MatrixSpace(GF(randPrime), samples, dim)
M2Test = MatrixSpace(GF(randPrime), dim, 1)
M3Test = MatrixSpace(GF(randPrime), samples, 1)

# Fix the error and (binary) secret
# secret = M2Test([GF(randPrime).random_element() for i in range(dim)])
secret = M2Test(np.random.randint(q, size=(n,1)))
error_bools = np.random.binomial(1,expError/samples,size=(samples,1))
numErrors = int(np.sum(error_bools))

# while (samples - numErrors < dim):
#     error_bools = np.random.binomial(1,expError/samples,size=(samples,1))
#     numErrors = int(np.sum(error_bools))

# Generate random LPN instance
def gen_lpn(m,n,q,s,errorIndicator,numErr):
    A = M1Test([[GF(q).random_element() for i in range(n)] for j in range(m)])
    errors = [GF(q).random_element()*errorIndicator[i] for i in range(m)]
    B = M3Test(A*s) + M3Test(errors)
    return (A,B,numErr)

# Fixed LPN Instance
(A, B, numErrors) = gen_lpn(samples, dim, randPrime, secret,  error_bools, numErrors)

print("Number of samples: ", samples)
print("Number of variables: ", samples)
print("Error count: ", numErrors)

29
Number of samples:  5
Number of variables:  5
Error count:  2


In [5]:
# Make a coefficient vector for equation
def getCoeffs(p,convert):
    res = [0 for i in range(len(convert))]
    for (coeff,v) in p:
        res[convert[str(v)]] = coeff
    return res

# # Make a coefficient vector for equation BUT
# # We know some monomials are zero, so we can set their coeffcients to 0
def getCoeffs(p,convert,zeroed_out=set()):
    res = [0 for i in range(len(convert))]
    for (coeff,v) in p:
        if str(v) in zeroed_out: continue # the monomial v has been set to 0
        res[convert[str(v)]] = coeff
    return res

def get_all_monos(n,m,D,R):
    allVars = []
    for D_iter in range(D+1):
        degs = WeightedIntegerVectors(D_iter, [1 for i in range(n+m)])
        for deg in degs:
            res = 1
            for i in range(n+m):
                v = R('x'+str(i))
                res *= v**(deg[i])
            allVars.append(res)
    allVars.sort()
    return allVars
    

def remove_high_deg_monos(m,n,high_deg,k,R):
    zero_monos = set()
    elts = [n+i for i in range(m)]
    for i in range(k):
        mono = None
        while True:
            guess = np.random.choice(elts,size=high_deg,replace=False)
            mono = R(1)
            for j in guess:
                mono *= R('x'+str(j))
            mono = str(mono)
            if mono not in zero_monos: break
        zero_monos.add(mono)
    return zero_monos

def increase_zero_monos(zero_monos,k,D,R):
    d = D-k
    half_vars = get_all_monos(n,m,d,R)[1:]
    for i in zero_monos:
        for j in half_vars:
            mono = str(i*j)
            zero_monos.add(mono)
    return zero_monos
    
    

def determine_num_rand_eqns(n,m,d,numErrors,delta=0.4):
    alpha = log(m,n) - 1
    eps = 1 + alpha - log(numErrors,n)
    eps_prime = log(d,n) - eps
    num_rand_eqns = int(-log(1-delta) * 2**(n**eps_prime))
    print('k',num_rand_eqns)
    print('eps_prime',eps_prime)
    return num_rand_eqns
    
    
def checkMonomial(v,d,n,m):
    if v.degree() > d: return False
    return True

# Expansion by s_i's up to degree D
def expand(d,R,m,n,A,B,numErrors,rem_high_deg=False):
    # Maximal expansion degree
    D = d+2
    usedVars = set()
    
    # Produce all variables up to degree D
    # x_0,...,x_(n-1) are components of secret vector s
    # x_n,...,x_(n+m-1) are the alpha_i s
    allVars = get_all_monos(n,m,D,R)
    
    # Mapping of monomial to position in the ordering of all monomials
    convert = dict()
    for i in range(len(allVars)):
        convert[str(allVars[i])] = i
    
    # Populate Macaulay matrix
    macaulay_matrix = []
    for i in range(m):
        # Eqns. of the form: alpha_i * (<a_i, s> - b_i) = 0
        p = R(-B[i])
        for j in range(n):
            p += R(A[i][j]) * R('x'+str(j))
        p = R('x'+str(n+i)) * p
        
        # Eqn. of the form: alpha_i**2 - alpha_i
        p2 = R('x'+str(n+i))*R('x'+str(n+i)) - R('x'+str(n+i))
        for v in allVars:
            if checkMonomial(v,d,n,m):
                temp = v*p
                res = getCoeffs(temp,convert)
                macaulay_matrix.append(res)

                temp = v*p2
                res = getCoeffs(v*p2,convert)
                macaulay_matrix.append(res)

    # Polynomial that gueses number of errors
    # (m - t) - \sum_{i=1}^m \alpha_i = 0
    p3 = R(numErrors - m)
    for i in range(m):
        p3 += R('x'+str(n+i))
    for v in allVars:
        if checkMonomial(v,d,n,m):
            temp = v*p3
            res = getCoeffs(temp,convert)
            macaulay_matrix.append(res)
    if rem_high_deg:
        high_deg = d # SET APPROPRIATELY
        k = determine_num_rand_eqns(n,m,d,numErrors,delta=0.1) # SET APPROPRIATELY
        zero_monos = remove_high_deg_monos(m,n,high_deg,k,R)
        zero_monos = increase_zero_monos(zero_monos,k,D,R)
        for mono in zero_monos:
            macaulay_matrix.append(getCoeffs(mono,convert))
        
        

    macaulay_matrix = np.array(macaulay_matrix)

    # in row of variables, the first variable is just the constant 1
    row = np.zeros(shape=(1,len(macaulay_matrix[0])))
    row[0][0] = 1
    macaulay_matrix = np.append(row,macaulay_matrix,axis=0)

    numEqns = len(macaulay_matrix)
    numVars = len(macaulay_matrix[0])

    M4 = MatrixSpace(R, numEqns, numVars, sparse=True)
    macaulay_matrix = M4(macaulay_matrix)
    rank = macaulay_matrix.rank()
    return [rank,numVars]

In [6]:
GFqX = PolynomialRing(GF(randPrime), samples + dim, 'x')
mac_degs = list(range(3,4))
for mac_deg in mac_degs:
    rank, numVars = expand(mac_deg, GFqX, samples, dim, A, B, numErrors, rem_high_deg=True)
    print("Degree of Expansion: ", mac_deg)
    print("Rank: ", rank, "\nNumber of variables: ", numVars)
    print("Ratio: ", rank/numVars)

k 0
eps_prime 0.16595623285353056


KeyboardInterrupt: 